#1. Install Library Needed

In [1]:
!pip install flask_ngrok
!pip install easyocr
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70.8 MB 21 kB/s 
     |████████████████████████████████| 47.6 MB 90 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 4.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=4cd38e78889c0aba1a4f43c653a5bfffad023c5c3dd84d89bb2e568f796474ea
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


#2. Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Orbit_Final_Project_ANPR_OCR/Deployment

/content/drive/MyDrive/Orbit_Final_Project_ANPR_OCR/Deployment


#3. Authenticate ngrok token

In [5]:
!ngrok authtoken '29pye4szs13NDncMCVgFjpoBCrL_7ghfGmHVQmxaDhGMPSgAG'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


#4. Run The Web

In [19]:
from flask import Flask, render_template, request
import os
from flask_ngrok import run_with_ngrok
from model import yolo_predictions, extract_text
import warnings
warnings.filterwarnings('ignore')

# webserver gateway interface
app = Flask(__name__)
run_with_ngrok(app)
BASE_PATH = os.getcwd()
UPLOAD_PATH = os.path.join(BASE_PATH, 'static/upload/')

@app.route('/', methods=['POST', 'GET'])
def index():
    if request.method == 'POST':
        upload_file = request.files['image_name']
        filename = upload_file.filename
        path_save = os.path.join(UPLOAD_PATH, filename)
        # Save uploaded image to google drive inside upload folder
        upload_file.save(path_save)
        print(filename)
        # Do predictions on the newly uploaded image
        text_ocr,text_ocr_processed = yolo_predictions(path_save, filename)

        return render_template('index.html', upload=True, upload_image=filename, text=text_ocr, text2=text_ocr_processed)
    return render_template('index.html', upload=False)

@app.route('/description', methods=['POST','GET'])
def desc():
    return render_template('desc.html')

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bbdb-35-189-169-253.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [18]:
!python --version

Python 3.7.13
